## Introducing the CQRS Pattern

CQRS is a Command Query Response Segregation pattern. This pattern separates read operations from update operations. The pattern is often used when read and update operations use a different data model. For example, the Account Details DynamoDB table would record each account transaction, while the Account Summary DynamoDB table would record a summary of transactions for each account. 

By using a CQRS pattern you would extract the records from the Account Details table, aggregate the records and write the records to the Account Summary table. An endpoint would be created for POST operation to account details while another endpoint would be used for GET operations on the account summary.

## The Scaling Challenge

Pattern 1 uses a DynamoDB stream to invoke a Lambda function that aggregates account detail records, transforming the records into an account summary. The transformation function writes directly to the Account Summary DynamoDB tables. This approach reduces latency; however, it does introduce a potential scaling problem. If the Account Detail DynamoDB table is configured to scale to a higher capacity, this will result in the DynamoDB stream scaling along with lambda function. If the Account Summary table is not configured to scale, the transformation function will start to timeout and return errors.

![architecture](../images/architecture_1.png "Architecture")

## The solution

Pattern 2 introduces a decoupling service such as an Amazon SQS queue. This will allow the two parts of the pattern to scale independently. The Account Summary table does not need to scale along with the Account Details table to meet the capacity demand.

![architecture](../images/architecture_2.png "Architecture")

**Jupyter Notebook**

This example uses 2 API Gateways, the gateway id needs to be set for both pattern examples.

```gwid_pattern_1 = …```

```gwid_pattern_2 = …```


In [29]:
#Set gateway id
gwid_pattern_1 = '...'
gwid_pattern_2 = '...'

**Pattern 1**

The script sends order records to the API Gateway. The gateway endpoint uses a mapping template to build a DynamoDB item object, then passes the object to the PutItem method as a parameter. DynamoDB will put the item into a stream which will invoke a Lambda function. The function will aggregate the order details and put a new item into the summary table. 

In [30]:
import boto3, requests, datetime
from random import randrange

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":3.5},{"size":"Medium","price":4.0},{"size":"Large","price":4.5},{"size":"x-Large","price":5.0}]

url = (f'https://{gwid_pattern_1}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

for i in range(5): #By increasing this counter you will generate more insteresting statistics for log insights
    accountid = 'a0'
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,            
            'coffeetype': coffeetype_list[randrange(4)],
            'coffeesize': coffeesize_list[coffeesize]["size"],
            'unitprice': coffeesize_list[coffeesize]["price"],
            'quantity': quantity
    }})

print(response)

<Response [200]>


**json formatter** - Run the following script to create a class which will be used to render json objects in a readable format.

In [31]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

The following script returns an aggregation of all the orders for an account. Amazon API Gateway can interface directly with Amazon DynamoDB. By passing an items key value, the API can call GetItem without having to invoke a Lambda function.

In [32]:
url_1 = (f'https://{gwid_pattern_1}.execute-api.ap-southeast-2.amazonaws.com/prod/a0')
response_get_1 = requests.get(url_1)
RenderJSON(response_get_1.json())

**Pattern 2**

The script sends order records to the API Gateway. The gateway endpoint uses a mapping template to build a DynamoDB item object, then passes the object to the PutItem method as a parameter. DynamoDB will put the item into a stream which will invoke a Lambda function. The function will aggregate the order details and send the records to an Amazon SQS queue. By introducing a decoupling service like SQS, the transformation function can scale up and process the items in DynamoDB stream without being restricted by the capacity on the destination table


In [33]:
import boto3, requests, datetime
from random import randrange

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":3.5},{"size":"Medium","price":4.0},{"size":"Large","price":4.5},{"size":"x-Large","price":5.0}]

url = (f'https://{gwid_pattern_2}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

for i in range(5): #By increasing this counter you will generate more insteresting statistics for log insights
    accountid = 'a0'
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,            
            'coffeetype': coffeetype_list[randrange(4)],
            'coffeesize': coffeesize_list[coffeesize]["size"],
            'unitprice': coffeesize_list[coffeesize]["price"],
            'quantity': quantity
    }})

print(response)

<Response [200]>


The following script returns an aggregation of all the orders for an account. Amazon API Gateway can interface directly with Amazon DynamoDB. By passing an items key value, the API can call GetItem without having to invoke a Lambda function.

In [36]:
url_2 = (f'https://{gwid_pattern_2}.execute-api.ap-southeast-2.amazonaws.com/prod/a0')
response_get_2 = requests.get(url_2)
RenderJSON(response_get_2.json())